***GENERATED CODE FOR highestincomeprediction PIPELINE***
***DON'T EDIT THIS CODE***

**CONNECTOR FUNCTIONS TO READ DATA FROM DATABRICKS FILESYSTEM**

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class DBFSConnector:

    def fetch(spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv(eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


**TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA**

In [ ]:
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import OneHotEncoderEstimator
import json
from pyspark.sql.functions import col, round
from pyspark.sql.functions import round, col
from pyspark.sql.functions import col, udf
from pyspark.sql.types import *
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import DoubleType, IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def to_array(col):
    def to_array_(v):
        return v.toArray().tolist()
    return udf(to_array_, ArrayType(DoubleType()))(col)


def OHETransform(df, params, transformationData={}):
    feature = params["feature"]
    dfReturn = df
    dfReturn = dfReturn.fillna({feature: ''})
    outcolS = feature + "_toint"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcolS, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcolS).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcolS, dfReturn[outcolS].cast(IntegerType()))
        dfReturn = changed_type_df

    outcol = feature + "_onehotencoder"
    encoder = OneHotEncoderEstimator(inputCols=[outcolS], outputCols=[
                                     outcol], handleInvalid="keep")
    model = encoder.fit(dfReturn)
    dfReturn = model.transform(dfReturn)
    dfReturn = dfReturn.drop(outcolS)
    dfReturn = dfReturn.withColumn("final_col", to_array(dfReturn[outcol])) \
        .select(dfReturn.schema.names + [col("final_col")[0]])
    dfReturn = dfReturn.drop(outcol).withColumnRenamed("final_col[0]", outcol)
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[outcol], 2))

    return dfReturn


def vectorAssemblerTransform(df, param):

    dfReturn = df

    if (type(param) == str):
        outcol = param + "_vector"
        assembler = VectorAssembler(inputCols=[param], outputCol=outcol)
        dfReturn = assembler.transform(dfReturn)
        return dfReturn

    if (type(param) == list):
        vecAssembler = VectorAssembler(inputCols=param, outputCol="features")
        new_df = vecAssembler.transform(df)
        return new_df


def standardScalarTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: '0.0'})
    scalarFlags = transformationData["std_scalar"]
    if scalarFlags["mean_flag"]:
        stdflag = False
        meanflag = True
    elif scalarFlags["std_flag"]:
        stdflag = True
        meanflag = False

    outcol = feature + "_standardscalar"

    featureVector = feature + "_vector"
    dfReturn = vectorAssemblerTransform(dfReturn, feature)

    standardscale = StandardScaler(inputCol=featureVector, outputCol=outcol, withStd=stdflag,
                                   withMean=meanflag)
    scaledata = standardscale.fit(dfReturn).transform(dfReturn)
    dfReturn = scaledata.withColumn("final_col", to_array(scaledata[outcol]))\
        .select(scaledata.schema.names + [col("final_col")[0]])

    dfReturn = dfReturn.drop(outcol).drop(featureVector)\
        .withColumnRenamed("final_col[0]", outcol)
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[outcol], 2))

    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Occupation', 'transformation_label': 'String Indexer'}, {'feature_label': 'Occupation', 'transformation_label': 'One Hot Encoding'}], 'feature': 'Occupation', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '215', 'mean': '', 'stddev': '', 'min': 'Accountants and auditors', 'max': 'Writers and authors', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}, {'transformation': 'One Hot Encoding', 'selectedAsDefault': 0}]}, {'feature_label': 'Occupation', 'transformation_label': 'String Indexer'})
        transformationDF = OHETransform(transformationDF, {'transformationsData': [{'feature_label': 'Occupation', 'transformation_label': 'String Indexer'}, {'feature_label': 'Occupation', 'transformation_label': 'One Hot Encoding'}], 'feature': 'Occupation', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '215', 'mean': '', 'stddev': '', 'min': 'Accountants and auditors', 'max': 'Writers and authors', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}, {'transformation': 'One Hot Encoding', 'selectedAsDefault': 0}]}, {'feature_label': 'Occupation', 'transformation_label': 'One Hot Encoding'})
        transformationDF = transformationDF.drop('Occupation')
        transformationDF = standardScalarTransform(transformationDF, {'transformationsData': [{'feature_label': 'M_workers', 'std_scalar': {'mean_flag': 'True', 'std_flag': 'False'}, 'transformation_label': 'Standard Scalar'}], 'feature': 'M_workers', 'transformation': [
            {'transformation': 'Standard Scalar', 'selectedAsDefault': 1}], 'type': 'numeric', 'replaceby': 'mean', 'selected': 'True', 'stats': {'count': '215', 'mean': '176.37', 'stddev': '493.1', 'min': '0', 'max': '5548', 'missing': '0'}}, {'feature_label': 'M_workers', 'std_scalar': {'mean_flag': 'True', 'std_flag': 'False'}, 'transformation_label': 'Standard Scalar'})
        transformationDF = transformationDF.drop('M_workers')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'M_weekly', 'transformation_label': 'String Indexer'}], 'feature': 'M_weekly', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '215', 'mean': '1008.11', 'stddev': '365.43', 'min': '1001', 'max': 'Na', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'M_weekly', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('M_weekly')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'F_weekly', 'transformation_label': 'String Indexer'}], 'feature': 'F_weekly', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '215', 'mean': '797.96', 'stddev': '286.9', 'min': '1025', 'max': 'Na', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'F_weekly', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('F_weekly')
        transformationDF = standardScalarTransform(transformationDF, {'transformationsData': [{'feature_label': 'All_workers', 'std_scalar': {'mean_flag': 'False', 'std_flag': 'True'}, 'transformation_label': 'Standard Scalar'}], 'feature': 'All_workers', 'transformation': [
            {'transformation': 'Standard Scalar', 'selectedAsDefault': 1}], 'type': 'numeric', 'replaceby': 'mean', 'selected': 'True', 'stats': {'count': '215', 'mean': '309.68', 'stddev': '746.03', 'min': '0', 'max': '7551', 'missing': '0'}}, {'feature_label': 'All_workers', 'std_scalar': {'mean_flag': 'False', 'std_flag': 'True'}, 'transformation_label': 'Standard Scalar'})
        transformationDF = transformationDF.drop('All_workers')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'All_weekly', 'transformation_label': 'String Indexer'}], 'feature': 'All_weekly', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '215', 'mean': '914.07', 'stddev': '329.22', 'min': '1001', 'max': 'Na', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'All_weekly', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('All_weekly')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


**AUTOML FUNCTIONS**

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = df.drop(label, axis=1).values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': sparkDF.columns}
    return data


**READING DATAFRAME**

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

%run highestincomepredictionHooks.ipynb
try:
	sourcePreExecutionHook()

	highestincomepredictiondbfs = DBFSConnector.fetch(spark, "{'url': '/Demo/PredictHighestIncomeTrain.csv', 'file_type': 'Delimeted', 'delimiter': ',', 'is_header': 'Use Header Line'}")

	sourcePostExecutionHook(highestincomepredictiondbfs)

except Exception as ex: 
	logging.error(ex)


**TRANSFORMING DATAFRAME**

In [ ]:
%run highestincomepredictionHooks.ipynb
try:
	transformationPreExecutionHook()

	highestincomepredictionautofe = TransformationMain.run(highestincomepredictiondbfs,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Occupation", "transformation_label": "String Indexer"}, {"feature_label": "Occupation", "transformation_label": "One Hot Encoding"}], "feature": "Occupation", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "215", "mean": "", "stddev": "", "min": "Accountants and auditors", "max": "Writers and authors", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}, {"transformation": "One Hot Encoding", "selectedAsDefault": 0}]}, {"transformationsData": [{"feature_label": "M_workers", "std_scalar": {"mean_flag": "True", "std_flag": "False"}, "transformation_label": "Standard Scalar"}], "feature": "M_workers", "transformation": [{"transformation": "Standard Scalar", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "215", "mean": "176.37", "stddev": "493.1", "min": "0", "max": "5548", "missing": "0"}}, {"transformationsData": [{"feature_label": "M_weekly", "transformation_label": "String Indexer"}], "feature": "M_weekly", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "215", "mean": "1008.11", "stddev": "365.43", "min": "1001", "max": "Na", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "F_workers", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "215", "mean": "133.25", "stddev": "343.07", "min": "0", "max": "2104", "missing": "0"}}, {"transformationsData": [{"feature_label": "F_weekly", "transformation_label": "String Indexer"}], "feature": "F_weekly", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "215", "mean": "797.96", "stddev": "286.9", "min": "1025", "max": "Na", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "All_workers", "std_scalar": {"mean_flag": "False", "std_flag": "True"}, "transformation_label": "Standard Scalar"}], "feature": "All_workers", "transformation": [{"transformation": "Standard Scalar", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "215", "mean": "309.68", "stddev": "746.03", "min": "0", "max": "7551", "missing": "0"}}, {"transformationsData": [{"feature_label": "All_weekly", "transformation_label": "String Indexer"}], "feature": "All_weekly", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "215", "mean": "914.07", "stddev": "329.22", "min": "1001", "max": "Na", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"feature": "Occupation_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "215", "mean": "107.0", "stddev": "62.21", "min": "0.0", "max": "214.0", "missing": "0"}}, {"feature": "M_weekly_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "215", "mean": "22.59", "stddev": "31.24", "min": "0.0", "max": "98.0", "missing": "0"}}, {"feature": "F_weekly_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "215", "mean": "11.56", "stddev": "20.28", "min": "0.0", "max": "70.0", "missing": "0"}}, {"feature": "All_weekly_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "215", "mean": "33.94", "stddev": "39.61", "min": "0.0", "max": "120.0", "missing": "0"}}, {"feature": "Occupation_onehotencoder_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "selected": "True", "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "stats": {"count": "215", "mean": "0.0", "stddev": "0.07", "min": "0.0", "max": "1.0", "missing": "0"}}, {"feature": "M_workers_standardscalar_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "selected": "True", "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "stats": {"count": "215", "mean": "0.0", "stddev": "493.1", "min": "-176.367441860465", "max": "5371.632558139535", "missing": "0"}}, {"feature": "All_workers_standardscalar_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "selected": "True", "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "stats": {"count": "215", "mean": "0.0", "stddev": "746.03", "min": "-309.6790697674417", "max": "7241.320930232559", "missing": "0"}}]}))

	transformationPostExecutionHook(highestincomepredictionautofe)

except Exception as ex: 
	logging.error(ex)


**TRAIN MODEL**

In [ ]:
%run highestincomepredictionHooks.ipynb
try:
	mlPreExecutionHook()

	dataAutoML=functionRegression(highestincomepredictionautofe, ["F_workers", "Occupation_stringindexer", "M_weekly_stringindexer", "F_weekly_stringindexer", "Occupation_onehotencoder", "M_workers_standardscalar", "All_workers_standardscalar"], "All_weekly_stringindexer")

	mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


**PREDICT ON TRAINED MODEL**

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

